In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm

In [2]:
example = os.listdir("images")[0].split(".")[0]
image = plt.imread(f"images/{example}.png")
label = np.array(open(f"labels/{example}.txt").read().split()).astype(np.float16)[1:] / 2

# 1. Augmentation Functions

In [3]:
def rotate(image, label, left=True):
    
    def rot90(image, label):
        RCX, RCY, RW, RH= label
        rcx, rcy, rw, rh = RCY, 1 - RCX, RH, RW
        return np.rot90(image), (rcx, rcy, rw, rh)
    
    def rot270(image, label):
        RCX, RCY, RW, RH= label
        rcx, rcy, rw, rh = 1 - RCY, RCX, RW, RH
        return np.rot90(image, 3), (rcx, rcy, rw, rh)
    
    if left:
        result =  rot90(image, label)
    else:
        result =  rot270(image, label)
    return result

In [4]:
def flip(image, label, horizontal=True):
    def vflip(image, label):
        RCX, RCY, RW, RH = label
        rcx, rcy, rw, rh = RCX, 1-RCY, RW, RH
        return np.flip(image, axis=0), (rcx, rcy, rw, rh)

    def hflip(image, label):
        RCX, RCY, RW, RH = label
        rcx, rcy, rw, rh = 1-RCX, RCY, RW, RH
        return np.flip(image, axis=1), (rcx, rcy, rw, rh)
    
    if horizontal:
        result =  hflip(image, label)
    else:
        result =  vflip(image, label)
    return result

In [5]:
def sharpen(image, label, execute):
    if execute:
        kernel = np.array([[-1,-1,-1,-1,-1],[-1,2,2,2,-1],[-1,2,8,2,-1],[-1,2,2,2,-1],[-1,-1,-1,-1,-1]])/8.
        sharpened = cv2.filter2D(image,-1,kernel)
        return sharpened, label
    else:
        return image, label

# 2. Random operations

In [6]:
SAVE_PATH = "aug-images/"

In [7]:
def augment(image, label, random_code):
    R, F, S = random_code
    image, label = rotate(image, label, R)
    image, label = flip(image, label, F)
    image, label = sharpen(image, label, S)
    return image, label

In [12]:
def save(image, label, name):
    img_save_path = SAVE_PATH + f"{name}.png"
    label_save_path = SAVE_PATH + f"{name}.txt"
    
    cv2.imwrite(img_save_path, image)
    rcx, rcy, rw, rh = label
    line = f"0 {rcx} {rcy} {rw} {rh}"
    with open(label_save_path, "w") as labelfile:
        labelfile.write(line)

In [18]:
names = [fname.split("/")[1][:-4] for fname in open("meta/train.txt", "r").read().split("\n")[:-1]]

In [22]:
for name in tqdm(names):
    # read image & label file
    image = cv2.imread(f"images/{name}.png", cv2.IMREAD_GRAYSCALE)
    label = np.array(open(f"labels/{name}.txt").read().split()).astype(np.float16)[1:] / 2
    
    # create 2 images & 2 labels
    prior_code = [0,0,0]
    for i in range(2):
        while True:
            random_code = np.random.randint(0, 2, 3)
            if (random_code != prior_code).any():
                break
        image, label = augment(image, label, random_code)
        
        save_name = name + f"_{i}"
        save(image, label, save_name)
        prior_code = random_code[:]

In [23]:
len(os.listdir("aug-images")) # 3,600 images, 3,600 labels

7200

In [25]:
images = set([fname.split(".")[0] for fname in os.listdir("aug-images") if fname.split(".")[-1]=="png"])
labels = set([fname.split(".")[0] for fname in os.listdir("aug-images") if fname.split(".")[-1]=="txt"])

In [26]:
images == labels

True

# 3. Write metafiles

In [29]:
train_list = ["aug-images/" + name + ".png" for name in images]
with open("meta/aug-train.txt", "w") as metafile:
    for fname in tqdm(train_list):
        metafile.write(fname + "\n")

In [32]:
train_list[:10]

['aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-001898_1.png',
 'aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-001976_0.png',
 'aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-000438_1.png',
 'aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-001138_1.png',
 'aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-000121_0.png',
 'aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-001494_1.png',
 'aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-000892_0.png',
 'aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-000181_1.png',
 'aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-001995_1.png',
 'aug-images/000-pgan-cxr_abnormal-preset-v2-1gpu-fp32-network-snapshot-014000-001183_1.png']

In [31]:
len(train_list)

3600